# **3-D FDTD (PEC Boundary)**

### *Code*

In [ ]:
import meep as mp
import numpy as np
import matplotlib.pyplot as plt

# Constants
c = 3e+8                                                # speed of light (m/s)

# FDTD Variables
dx, dy, dz = 1e-3, 1e-3, 1e-3                           # spatial step (m)
C = 0.99                                                # Courant factor
dt = C / (c * np.sqrt((1/dx)**2+(1/dy)**2+(1/dz)**2))   # time step (s)
sx, sy, sz = 30, 40, 10                                 # simulation cell size

# Gaussian Source Coefficients
E_0 = 1
n_T = 3                                     # time delay
bw = 15e+9                                  # bandwidth (Hz)
T = np.sqrt(2 * np.log(2)) / (np.pi * bw)   # (s)

# Create Simulation
cell = mp.Vector3(sx,sy,sz)

geometry = [mp.Block(cell,
                     center=mp.Vector3(),
                     material=mp.perfect_electric_conductor)]

sources = [mp.Source(mp.GaussianSource(frequency=bw, fwidth=1/T, start_time=n_T*T),   # pulse = E_0 * np.exp(-(((n * dt - n_T * T) / T))**2)
                     component=mp.Ez,
                     center=mp.Vector3(-cell.x/2 + 3*dx, -cell.y/2 + 3*dy, -cell.z/2 + 3*dz))]

resolution = 10   # ??

sim = mp.Simulation(cell_size=cell, 
                    geometry=geometry, 
                    sources=sources, 
                    resolution=resolution)

# Observation
pos = mp.Vector3(-cell.x/2 + 12*dx, -cell.y/2 + 3*dy, -cell.z/2 + 10*dz)
ez_data = []
def ez_probe(sim):
    ez_data.append(sim.get_field_point(mp.Ez, pos))

# Run Simulation for 8192 time steps & Save Ez at every time step from the observation position
sim.run(mp.at_every(dt, ez_probe), until=8192*dt)

print("Ez:")
print(ez_data)

# Calculate the resonance frequencies using DFT (range: 5 - 30 GHz)

# Compare the FDTD result w/ analytical result (refer Balanis pdf)



-----------
Initializing structure...
time for choose_chunkdivision = 1.50204e-05 s
Working in 3D dimensions.
Computational cell is 30 x 40 x 10 with resolution 1
     block, center = (0,0,0)
          size (30,40,10)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (-1e+20,-1e+20,-1e+20)
time for set_epsilon = 0.078882 s
-----------
run 0 finished at t = 0.5 (1 timesteps)
Ez:
[0j, 0j]


### *Result*